In [1]:
cd(@__DIR__)
ENV["CELLLISTMAP_8.3_WARNING"] = "false"
include("../src/juliaEAM.jl")
include("../src/lammpsIO.jl")

using Pkg
Pkg.activate(".")

using Printf
using AtomsCalculators
# using ASEconvert # use this PR:https://github.com/mfherbst/ASEconvert.jl/pull/17, Pkg.add(url="https://github.com/tjjarvinen/ASEconvert.jl.git", rev="atomscalculators")
using Unitful: Å, nm
# using PythonCall
# ENV["PYTHON"] = "/SNS/users/ccu/miniconda3/envs/analysis/bin/python"
# install the following packages in julia REPL
# using CondaPkg
# CondaPkg.add_pip("IPython")
# CondaPkg.add_pip("nglview")
using StaticArrays: SVector
# using GLMakie
using Molly
# using Zygote
using LinearAlgebra
# import Interpolations:cubic_spline_interpolation, linear_interpolation, interpolate, BSpline, Cubic, scale, Line, OnGrid, extrapolate, Gridded, extrapolate, Flat
using DelimitedFiles
using UnitfulAtomic
import PeriodicTable
using ProgressMeter
using Random

  Activating new project at `~/Documents/ABCD_J/EAM`
  Activating new project at `~/Documents/ABCD_J/EAM`


In [3]:
al_LatConst = 4.0495/10 # nm
atom_mass = 26.9815u"u"  # Atomic mass of aluminum in grams per mole

26.9815 u

In [4]:
eam = EAM()
fname = "Al99.eam.alloy"
read_potential!(eam, fname)

### Define customized interaction type in `AtomsCalculators`

In [6]:
struct EAMInteractionJulia
    calculator::Any  # Holds the ASE EAM calculator reference
    f_energy::Any    # Holds the energy function
    f_forces::Any    # Holds the forces function
end

### Initialize System and run simulator

In [7]:
### Initialize System and run simulatoreamJulia = EAMInteractionJulia(eam,calculate_energy,calculate_forces)
function initialize_system_dump(;loggers=(coords=CoordinateLogger(1),),filename_dump="")
    n_atoms, box_size, coords_molly, attypes = lmpDumpReader(filename_dump)
    molly_atoms = [Molly.Atom(index=i, charge=0, mass=atom_mass, 
                    #   σ=2.0u"Å" |> x -> uconvert(u"nm", x), ϵ=ϵ_kJ_per_mol
                    ) for i in 1:length(coords_molly)]
    # Specify boundary condition
    boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])

    atom_positions_init = copy(coords_molly)
    molly_atoms_init = copy(molly_atoms)
    # Initialize the system with the initial positions and velocities
    system_init = Molly.System(
    atoms=molly_atoms_init,
    atoms_data = [AtomData(element="Al", atom_type=string(attypes[ia])) for (ia,a) in enumerate(molly_atoms_init)],
    coords=atom_positions_init,  # Ensure these are SVector with correct units
    boundary=boundary_condition,
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    neighbor_finder = DistanceNeighborFinder(
    eligible=trues(length(molly_atoms_init), length(molly_atoms_init)),
    n_steps=1e3,
    dist_cutoff=7u"Å"),
    loggers=loggers,
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/Å"  # Ensure these units are correctly specified
    )
    return system_init
end

initialize_system_dump (generic function with 1 method)

In [8]:
filename_dump = "./LAMMPS/out_vacancy_cube8.dump"
molly_system = initialize_system_dump(filename_dump = filename_dump)

System with 2047 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[32.3974677930295 Å, 32.3974677930295 Å, 32.3974677930295 Å])

In [70]:
"""
    calculate_atomstress(eam::EAM, sys::Molly.System, neighbors_all::Vector{Vector{Int}})
    
Calculate the atomic stress for a given EAM potential, system, and neighbor list.

# Arguments
- `eam::EAM`: The EAM potential.
- `sys::Molly.System`: The system.
- `neighbors_all::Vector{Vector{Int}}`: The neighbor list for each atom.

# Returns
- `stresses_particle::Array{Float64, 3}`: The atomic stress tensor for each atom.

"""
function calculate_atomstress(eam::EAM, sys::Molly.System, neighbors_all::Vector{Vector{Int}})
    coords = [ustrip(coord) for coord in sys.coords]
    boundary = @SVector [ustrip(sys.boundary[i]) for i in 1:3]
    return calculate_atomstress(eam, coords, boundary, neighbors_all)
end

"""
    calculate_atomstress(eam::EAM, coords::Vector{SVector{3, Float64}}, boundary::SVector{3, Float64}, neighbors_all::Vector{Vector{Int}})
    
Calculate the atomic stress for a given EAM potential, coordinates, boundary, and neighbor list.

# Arguments
- `eam::EAM`: The EAM potential.
- `coords::Vector{SVector{3, Float64}}`: The coordinates of the atoms.
- `boundary::SVector{3, Float64}`: The boundary of the system.
- `neighbors_all::Vector{Vector{Int}}`: The neighbor list for each atom.

# Returns
- `stresses_particle::Array{Float64, 3}`: The atomic stress tensor for each atom.

"""
function calculate_atomstress(eam::EAM, coords::Vector{SVector{3, Float64}}, boundary::SVector{3, Float64}, neighbors_all::Vector{Vector{Int}})
    n_threads::Int = 1
    
    ## for single element system, only one type is considered
    typelist::Vector{Int} = [1]
    i_type::Int = 1 in typelist ? indexin(1, typelist)[1] : error("1 not found in typelist")
    d_electron_density_i = eam.d_electron_density[i_type]
    
    # preallocate
    # initialize forces_particle
    # forces_particle::Matrix{Float64} = zeros(length(coords),3)
    stresses_particle::Array{Float64, 3} = zeros(length(coords), 3, 3)
    
    # initialize total_density
    total_density::Vector{Float64} = zeros(length(coords))

    r_all::Vector{Matrix{Float64}} = []
    d_all::Vector{Vector{Float64}} = []

    n_neighbors_all::Vector{Int} = [length(neighbors_all[i]) for i in 1:length(coords)]
    n_neighbors_max::Int = maximum(n_neighbors_all)

    r_i::Matrix{Float64} = zeros(n_neighbors_max,3)
    d_i::Vector{Float64} = zeros(n_neighbors_max)
    for i::Int in 1:length(coords)
        # i_type::Int = indexin(1, typelist)[1]
        
        # neighbors = get_neighbors(neig, i)
        neighbors::Vector{Int} = neighbors_all[i]    
        if isempty(neighbors)
            continue
        end

        n_neighbors::Int = length(neighbors)
        coords_i = coords[i]
    
        # distance between atom i and its neighbors
        # r_i::Matrix{Float64} = zeros(n_neighbors,3)
        # d_i::Vector{Float64} = zeros(n_neighbors)
        for (index_j::Int, j::Int) in enumerate(neighbors)
            r_ij = (vector(coords_i, coords[j], boundary)) # Å
            d_ij = sqrt(sum(r_ij.^2))
            r_i[index_j,1:3] = r_ij
            d_i[index_j] = minimum((d_ij,eam.r[end]))
        end

        push!(r_all, r_i[1:n_neighbors,1:3])
        push!(d_all, d_i[1:n_neighbors])
    
        for j_type::Int in 1:eam.Nelements # iterate over all types
            # use = get_type(neighbors, typelist) .== j_type # get the index of the neighbors with type j
            # if !any(use)
            #     continue
            # end
            # d_use = d_i[use]
            d_use = d_all[i]
    
            density = sum(eam.electron_density[j_type].(d_use)) # electron density
            total_density[i] += density # total electron density around atom i
        end
    end
    
    # calculate stresses on particles
    for i::Int in 1:length(coords)
        # i_type::Int = indexin(1, typelist)[1]
            
        # neighbors = get_neighbors(neig, i)
        neighbors::Vector{Int} = neighbors_all[i]
        if isempty(neighbors)
            continue
        end
        n_neighbors::Int = length(neighbors)
        coords_i = coords[i]
    
        r_i = r_all[i]
        d_i = d_all[i]
        
        # derivative of the embedded energy of atom i
        d_embedded_energy_i::Float64 = eam.d_embedded_energy[i_type].(total_density[i])
    
        ur_i = r_i
    
        # unit directional vector
        ur_i ./= d_i
        
        sigma_i::Matrix{Float64} = zeros(3, 3)
        for j_type::Int in 1:eam.Nelements
            # use = get_type(neighbors, typelist) .== j_type # get the index of the neighbors with type j
            # if !any(use)
            #     continue
            # end
            # r_use = r_i[use]
            # d_use = d_i[use]
            # ur_use = ur_i[use, :]
            # neighbors_use = neighbors[use]
            r_use = r_i
            d_use = d_i
            ur_use::Matrix{Float64} = ur_i[:,:]
            neighbors_use = neighbors
    
            total_density_j = total_density[neighbors_use]
            
            scale::Vector{Float64} = eam.d_phi[i_type, j_type].(d_use)
            scale .+= (d_embedded_energy_i .* eam.d_electron_density[j_type].(d_use)) 
            scale .+= (eam.d_embedded_energy[j_type].(total_density_j) .* d_electron_density_i.(d_use))
            
            # forces_particle[i, :] .+= (ur_use' * scale) # get pairwise force here
            forces_particle_ij = ur_use .* scale
            # print(size(forces_particle_ij))

            # Calculate the outer product and add it to the stress tensor
            for j in 1:size(forces_particle_ij, 1)
                sigma_i .+= r_i[j, :] * transpose(forces_particle_ij[j, :])
            end
            stresses_particle[i,:,:] = sigma_i
        end
    end

    return stresses_particle
end

calculate_atomstress (generic function with 2 methods)

In [83]:
neighbors_all = get_neighbors_all(molly_system)
stresses = calculate_atomstress(eam, molly_system, neighbors_all)
print(typeof(stresses))
function f_vm_stress(stresses)
    vm_stress = zeros(size(stresses)[1])
    for i in 1:size(stresses)[1]
        σ = stresses[i, :, :]
        σ_xx, σ_yy, σ_zz = σ[1, 1], σ[2, 2], σ[3, 3]
        σ_xy, σ_yz, σ_zx = σ[1, 2], σ[2, 3], σ[3, 1]
        vm_stress[i] = sqrt(0.5 * ((σ_xx - σ_yy)^2 + (σ_yy - σ_zz)^2 + (σ_zz - σ_xx)^2 + 6*(σ_xy^2 + σ_yz^2 + σ_zx^2)))
    end 

    return vm_stress
end
vm_stress = f_vm_stress(stresses)

Array{Float64, 3}

2047-element Vector{Float64}:
 8.416260535715272e-10
 6.081627916047773e-6
 6.081627916047776e-6
 6.081627916242058e-6
 5.4303555579053674e-6
 6.754858623730678e-6
 6.7548586237160425e-6
 6.510533840769375e-7
 2.2774234710903852e-5
 5.268384345152409e-6
 ⋮
 4.020913477427076e-5
 4.116597036557926e-5
 6.212874006989114e-6
 3.9529752978575874e-5
 3.952975297855346e-5
 4.301434029412511e-5
 3.105211615096748e-8
 3.6799130386866876e-5
 3.679913038678362e-5

In [84]:
size(stresses)

(2047, 3, 3)

In [89]:
"""
    lmpDumpWriter_prop(file, timestep, sys, fname_dump, property)

Write atomic data to a LAMMPS dump file with additional property.

# Arguments
- `file`: The file object to write the data to.
- `timestep`: The current timestep.
- `sys`: The system object containing atomic coordinates and properties.
- `fname_dump`: The filename of the dump file.
- `property`: An array of additional properties for each atom.
"""
function lmpDumpWriter_prop(file,timestep,sys,fname_dump,property)
    # open(fname_dump, "a") do file
    write(file, "ITEM: TIMESTEP\n")
    write(file, string(timestep)*"\n")
    write(file, "ITEM: NUMBER OF ATOMS\n")
    write(file, string(length(sys.coords))*"\n")
    write(file, "ITEM: BOX BOUNDS pp pp pp\n")
    write(file, "0 "*string(ustrip(sys.boundary[1]))*"\n")
    write(file, "0 "*string(ustrip(sys.boundary[2]))*"\n")
    write(file, "0 "*string(ustrip(sys.boundary[3]))*"\n")
    write(file, "ITEM: ATOMS id type xu yu zu property\n")
    for (i_c, coord) in enumerate(sys.coords)
        atomdata = sys.atoms_data[i_c]
        write(file, string(i_c)*" "*string(atomdata.atom_type)*" "*join(ustrip(coord)," ")*" "*string(property[i_c])*"\n")
    end
    # end
end

"""
    lmpDataWriter_prop(file, timestep, sys, fname_dump, property)

Write atomic data to a LAMMPS dump file with additional property.

# Arguments
- `file`: The file object to write the data to.
- `timestep`: The current timestep.
- `sys`: The system object containing atomic coordinates and properties.
- `fname_dump`: The filename of the dump file.
- `property`: An array of additional properties for each atom.
"""
function lmpDataWriter_prop(file,timestep,sys,fname_dump,property)
    # open(fname_dump, "a") do file
    n_types = length(unique([ad.atom_type for ad in molly_system.atoms_data]))
    write(file, "# LAMMPS data file written by lammpsIO.jl\n")       #1
    write(file, "\n")                                                #2
    write(file, string(length(sys.coords))*" atoms\n")               #3
    write(file, string(n_types)*" atom types\n")                     #4
    write(file, "\n")                                                #5
    write(file, "0 "*string(ustrip(sys.boundary[1]))*" xlo xhi\n")   #6
    write(file, "0 "*string(ustrip(sys.boundary[2]))*" ylo yhi\n")   #7
    write(file, "0 "*string(ustrip(sys.boundary[3]))*" zlo zhi\n")   #8
    write(file, "\n")                                                #9
    write(file, "Atoms  # atomic\n")                                 #10
    write(file, "\n")                                                #11
    for (i_c, coord) in enumerate(sys.coords)
        atomdata = sys.atoms_data[i_c]
        write(file, string(i_c)*" "*string(atomdata.atom_type)*" "*join(ustrip(coord)," ")*" "*string(property[i_c])*"\n")
    end
    # end
end



lmpDataWriter_prop

In [90]:
open("test_stress.dump", "w") do file
    lmpDumpWriter_prop(file,0,molly_system,"test_stress.dump",vm_stress)
end

open("test_stress.data", "w") do file
    lmpDataWriter_prop(file,0,molly_system,"test_stress.data",vm_stress)
end